In [ ]:
import mikeio
import pandas as pd
import numpy as np
import glob
import os
from mikeio.eum import EUMType, EUMUnit, ItemInfo
from mikecore.DfsFile import DataValueType
from mikeio import dfs0
import datetime

# Reading the data extraction location points information and setting the output file
L_df = pd.read_excel(r"G:\MJI\SKZ\Script\DFS2 to Wind\Data_Extraction_Points.xlsx")
output_folder_path = r"G:\MJI\SKZ\Script\DFS2 to Wind\Output"

# Defining the lists to store the location information
loc_list = []
x_coord = []
y_coord = []

# Iterating through each point from the file and extracting the required data
for location, x_co,y_co in zip(L_df['Location_ID'],L_df['X'],L_df['Y']):
    loc_list.append(location)
    x_coord.append(x_co)
    y_coord.append(y_co)


# Iterating through each point extracting the data
for i, j, k in zip(loc_list, x_coord, y_coord):
    
    location_ID = i
    grid_x = j
    grid_y = k
    
    r_df = pd.DataFrame()
    
    # Ierating through each wind file and extracting the data
    for file in glob.glob(r"\\10.0.0.33\New folder (D)\LTM\**\*.dfs2"):
        
        file_path_list = file.split(os.sep)
        dfs2_file_name = file_path_list[-1][:-5]

        ds = mikeio.read(file)

        Wind_X_comp = ds["Wind X-comp."]
        Wind_Y_comp = ds["Wind Y-comp."]

        points_data_x = Wind_X_comp.isel(x = grid_x, y= grid_y)
        points_data_y = Wind_Y_comp.isel(x = grid_x, y= grid_y)

        df_x = pd.DataFrame({'time': points_data_x.time, 'wind_x': points_data_x.values})
        df_y = pd.DataFrame({'time': points_data_y.time, 'wind_y': points_data_y.values})

        df = df_x.merge(df_y, how='left', on= 'time' )
        df['Wind (m/s)'] = np.sqrt((df_x['wind_x'])**2+(df_y['wind_y'])**2)

        # SAVING THE INDIVIDUAL WIND SPEED FILE

        # Setting the date/time column as index
        # and creating a dataframe with the rest of the columns
        df.set_index(pd.to_datetime(df['time']), inplace=True)
        v_df = df.iloc[:,1:]

        # Defining a list to store the dfs0 file item
        item_list = []

        # Iterating through all the column not equal to 'Date' column and
        # creating item for the dfs0 file and updating in the item list
        for col in v_df.columns:

            if col != 'time':

                # Defining item (wind speed) and unit based on type
                item = ItemInfo(col , EUMType.Wind_speed, EUMUnit.meter_per_sec, data_value_type = DataValueType.Instantaneous)

                # Updating the item in the item list
                item_list.append(item)


        # Setting the dfs0 file name and Saving the dataframe as dfs0 file
        dfs0_file_path = os.path.join(output_folder_path, '{}_{}.dfs0'.format(location_ID, dfs2_file_name))
        v_df.to_dfs0(dfs0_file_path, items = item_list, title='{} [{}, {}]'.format(location_ID, grid_x, grid_y))


        max_df = df.nlargest(1, 'Wind (m/s)')  # df.nlarges( n = Number of rows to return, Column name)
        r_df = pd.concat([r_df, max_df])
        
    #SAVING THE EACH CYCLONE MAXIMUM WIND SPEED           
    max_item_list = []
    
    r_df.reset_index(drop = True, inplace = True)
    r_df.sort_values(by='time',ascending=True, inplace = True)
    r_df.set_index(pd.to_datetime(r_df['time']), inplace=True)
    point_max_df = r_df.iloc[:,1:]
                     
    for col_name in point_max_df.columns:

        if col_name != 'time':

            # Defining item (wind speed) and unit based on type
            max_item = ItemInfo(col_name, EUMType.Wind_speed, EUMUnit.meter_per_sec, data_value_type = DataValueType.Instantaneous)

            # Updating the item in the item list
            max_item_list.append(max_item)

    # Setting the dfs0 file name and Saving the dataframe as dfs0 file
    point_max_dfs0_file_path = os.path.join(output_folder_path, '{}_Cyclone_Max_Wind_Speed.dfs0'.format(location_ID))
    point_max_df.to_dfs0(point_max_dfs0_file_path, items = max_item_list, 
                         title='{} [{}, {}]'.format(location_ID, grid_x, grid_y))